In [1]:
import numpy as np

In [2]:
fs = 250

In [3]:
from scipy.signal import butter, filtfilt, iirnotch

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def notch_filter(data, freq, fs, quality=30):
    nyq = 0.5 * fs
    freq = freq / nyq
    b, a = iirnotch(freq, quality)
    y = filtfilt(b, a, data)
    return y

In [4]:
def baseline_correction(data):
    # Baseline correction using the polynomial fit method
    # Fit a line to the data
    p = np.polyfit(np.arange(len(data)), data, 1)  # Polynomial fit of degree 1 (linear)
    # Subtract the fitted line from the data
    corrected_data = data - np.polyval(p, np.arange(len(data)))
    return corrected_data

In [5]:
from scipy.signal import welch

# Function to perform spectral analysis
def spectral_analysis(data, fs):
    # Compute the power spectral density using Welch's method
    f, Pxx = welch(data, fs=fs, nperseg=1024)
    return f, Pxx

In [6]:
alpha_band = (8, 13)
beta_band = (13, 30)

# Function to extract power in a specific frequency band
def extract_band_power(frequencies, power_spectrum, band):
    band_mask = (frequencies >= band[0]) & (frequencies <= band[1])
    band_power = np.sum(power_spectrum[band_mask])
    return band_power

In [7]:
from scipy.ndimage import uniform_filter1d

# Define the window size for the moving average (e.g., corresponding to about 1 second of data)
window_size = int(fs * 1)  # 1 second window, given the sampling rate (fs) is 250 Hz

# Function to calculate moving average using a uniform filter for simplicity
def moving_average(data, size):
    return uniform_filter1d(data, size=size, mode='reflect')

In [8]:
start_index = 3 * 250  # 3 seconds * 250 Hz
end_index = 5 * 250    # 5 seconds * 250 Hz

In [23]:
# Load the latest uploaded .npy file
latest_data = np.load(r'test\eb229854-1a14-4e1f-8e52-470edbec47c9.npy')

# Assuming the same configuration: first 8 channels, 250 Hz sampling rate
# Apply baseline correction, bandpass filter (0.5-30 Hz), and notch filter (60 Hz) to the latest data
latest_baseline_corrected_channels = np.array([baseline_correction(ch) for ch in latest_data[:, :8].T]).T
latest_bandpassed_channels = np.array([bandpass_filter(ch, 0.5, 30, fs) for ch in latest_baseline_corrected_channels.T]).T
latest_filtered_channels = np.array([notch_filter(ch, 60, fs) for ch in latest_bandpassed_channels.T]).T

# Calculate the beta power as a time series for each of the motor-related channels (C3, C4)
latest_beta_power_ts_c3 = bandpass_filter(latest_filtered_channels[:, 1], 7, 40, fs)  # C3
latest_beta_power_ts_c4 = bandpass_filter(latest_filtered_channels[:, 3], 7, 40, fs)  # C4

# Apply moving average to smooth the beta power time series
latest_smoothed_beta_c3 = moving_average(latest_beta_power_ts_c3**2, window_size)
latest_smoothed_beta_c4 = moving_average(latest_beta_power_ts_c4**2, window_size)

# Extract the task period from the latest data based on the known active period (3rd to 5th second)
latest_task_period_beta_c3 = latest_smoothed_beta_c3[start_index:end_index]
latest_task_period_beta_c4 = latest_smoothed_beta_c4[start_index:end_index]

# Calculate the average beta power during the task period for each channel
latest_avg_beta_power_c3 = np.mean(latest_task_period_beta_c3)
latest_avg_beta_power_c4 = np.mean(latest_task_period_beta_c4)

# Make a decision based on the beta power comparison:
# Considering a threshold to determine if it's resting or active based on average power being significantly lower
resting_threshold = 0.5 * (latest_avg_beta_power_c3 + latest_avg_beta_power_c4)
if latest_avg_beta_power_c3 < resting_threshold and latest_avg_beta_power_c4 > resting_threshold:
    latest_conclusion = "110."
elif latest_avg_beta_power_c4 < resting_threshold and latest_avg_beta_power_c3 > resting_threshold:
    latest_conclusion = "120"
elif latest_avg_beta_power_c3 > resting_threshold and latest_avg_beta_power_c4 > resting_threshold:
    latest_conclusion = "150"

latest_conclusion

'120'

In [16]:
#120 right arm